In [4]:
cd C:\Users\kvnzh\Desktop\Basketball

C:\Users\kvnzh\Desktop\Basketball


In [1]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from datetime import *

In [2]:
list_teams = ['ATL','BOS','BRK','CHI','CHO','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA','MIL','MIN',
              'NOP','NYK','OKC','ORL','PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS','NJN','NYA','NOH','NOK','CHA','CHH',
              'SEA','VAN','DNA','INA','SDC','BUF','KCK','KCO','SAA','NOJ','CAP','WSB']

team_mapping = {'ATL':'Atlanta Hawks',
                'BRK':'Brooklyn Nets',
                'BOS':'Boston Celtics',
                'CHO':'Charlotte Hornets',
                'CHI':'Chicago Bulls',
                'CLE':'Cleveland Cavaliers',
                'DAL':'Dallas Mavericks',
                'DEN':'Denver Nuggets',
                'DET':'Detroit Pistons',
                'GSW':'Golden State Warriors',
                'HOU':'Houston Rockets',
                'IND':'Indiana Pacers',
                'LAC':'Los Angeles Clippers',
                'LAL':'Los Angeles Lakers',
                'MEM':'Memphis Grizzlies',
                'MIA':'Miami Heat',
                'MIL':'Milwaukee Bucks',
                'MIN':'Minnesota Timberwolves',
                'NOP':'New Orleans Pelicans',
                'NYK':'New York Knicks',
                'OKC':'Oklahoma City Thunder',
                'ORL':'Orlando Magic',
                'PHI':'Philadelphia 76ers',
                'PHO':'Phoenix Suns',
                'POR':'Portland Trail Blazers',
                'SAC':'Sacramento Kings',
                'SAS':'San Antonio Spurs',
                'TOR':'Toronto Raptors',
                'UTA':'Utah Jazz',
                'WAS':'Washington Wizards'}

Geolocation = {'Philadelphia 76ers':[39.952583,-75.165222],
               'Oklahoma City Thunder':[35.467560,-97.516426],
               'Milwaukee Bucks':[43.038902,-87.906471],
               'Brooklyn Nets':[40.678177,-73.944160],
               'Memphis Grizzlies':[35.149532,-90.048981],
               'Miami Heat':[25.7741728,-80.19362],
               'Atlanta Hawks':[33.7489924,-84.3902644],
               'Cleveland Cavaliers':[41.5051613,-81.6934446],
               'New Orleans Pelicans':[29.9499323,-90.0701156],
               'Minnesota Timberwolves':[44.9788262,-93.274851],
               'Utah Jazz':[40.6593864,-111.8650787],
               'Denver Nuggets':[39.7392364,-104.9848623],
               'Dallas Mavericks':[32.7762719,-96.7968559],
               'Chicago Bulls':[41.8810409,-87.6711385],
               'Los Angeles Lakers':[34.0536909,-118.242766],
               'Los Angeles Clippers':[34.0536909,-118.242766],
               'Charlotte Hornets':[35.2272086,-80.8430827],
               'New York Knicks':[40.75054931640625,-73.99356079101562],
               'Sacramento Kings':[38.5810606,-121.4938951],
               'Boston Celtics':[42.3602534,-71.0582912],
               'Indiana Pacers':[39.7683331,-86.1583502],
               'Golden State Warriors':[37.8044557,-122.2713563],
               'Toronto Raptors':[43.6534817,-79.3839347],
               'Orlando Magic':[28.5421109,-81.3790304],
               'Detroit Pistons':[42.3315509,-83.0466403],
               'Phoenix Suns':[33.4484367,-112.0741417],
               'San Antonio Spurs':[29.4246002,-98.4951405],
               'Houston Rockets':[29.7589382,-95.3676974],
               'Washington Wizards':[38.8950368,-77.0365427],
               'Portland Trail Blazers':[45.5202471,-122.6741949]}

df = pd.DataFrame()
beg_year = int(input('What beg year: '))
end_year = int(input('What end year: '))

What beg year: 2018
What end year: 2019


# Figuring out which games to scrape 

In [5]:
df = pd.read_csv('CleanedUpData.csv')
df.head()

,Unnamed: 0,Start (ET),Away?,Opponent,TeamPts,OppPts,Team,TeamName,Location,PreviousLocation,DistanceFromPrevious,Datetime,DaysBetweenGames,SeasonAveragePPG,PTSoverAVG
0,0,19:00:00,@,Charlotte Hornets,91.0,109.0,ATL,Atlanta Hawks,"[35.2272086, -80.8430827]","[32.7762719, -96.7968559]",1493.035851,2017-10-20 19:00:00,1.937500,108.302469,-17.302469
1,1,15:30:00,@,Brooklyn Nets,104.0,116.0,ATL,Atlanta Hawks,"[40.678177, -73.94416]","[35.2272086, -80.8430827]",855.225043,2017-10-22 15:30:00,1.854167,108.302469,-4.302469
2,2,19:30:00,@,Miami Heat,93.0,104.0,ATL,Atlanta Hawks,"[25.7741728, -80.19362]","[40.678177, -73.94416]",1753.865770,2017-10-23 19:30:00,1.166667,108.302469,-15.302469
3,3,20:00:00,@,Chicago Bulls,86.0,91.0,ATL,Atlanta Hawks,"[41.8810409, -87.6711385]","[25.7741728, -80.19362]",1916.385380,2017-10-26 20:00:00,3.020833,108.302469,-22.302469
4,4,19:30:00,NaN,Denver Nuggets,100.0,105.0,ATL,Atlanta Hawks,"[33.7489924, -84.3902644]","[41.8810409, -87.6711385]",948.253647,2017-10-27 19:30:00,0.979167,108.302469,-8.302469


In [6]:
#InverseDictionary
inv_map = {v: k for k, v in team_mapping.items()}

In [7]:
#Map opponent names
df['OppName'] = df['Opponent'].map(inv_map)
df['OppName'].unique()

array(['CHO', 'BRK', 'MIA', 'CHI', 'DEN', 'MIL', 'PHI', 'HOU', 'CLE',
       'BOS', 'DET', 'WAS', 'NOP', 'SAC', 'SAS', 'LAC', 'NYK', 'TOR',
       'ORL', 'MEM', 'IND', 'OKC', 'DAL', 'POR', 'PHO', 'LAL', 'UTA',
       'MIN', 'GSW', 'ATL'], dtype=object)

In [8]:
#Temporary for BeautifulSoup
df[['Year', 'Rest']] = df['Datetime'].str.split('-', 1, expand=True)
df[['Mon', 'Rest']] = df['Rest'].str.split('-', 1, expand=True)
df[['Day', 'Rest']] = df['Rest'].str.split(' ', 1, expand=True)
df = df.drop(columns = ['Rest'])
df['ForBR'] = df['Year'] + df['Mon'] + df['Day'] + '0'

In [ ]:
#ForBS
df_new = pd.DataFrame()
for index, row in df.iterrows():
    try:
        if row['Away?'] != '@': #HomeGame
            r = get('https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fboxscores%2F'+ row['ForBR'] + row['Team'] + '.html&div=div_box-' + row['Team'] + '-game-basic')
            if r.status_code==200:
                    soup = BeautifulSoup(r.content, 'html.parser')
                    table = soup.find('table')
            df2 = pd.read_html(str(table))[0]
            df2 = df2.assign(Year = row['Year'],Team = row['Team'],ForBR = row['ForBR'])
            df_new = pd.concat([df_new,df2])
            sleep(3)
        else: #AwayGame
            r = get('https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fboxscores%2F'+ row['ForBR'] + row['OppName'] + '.html&div=div_box-' + row['Team'] + '-game-basic')
            if r.status_code==200:
                    soup = BeautifulSoup(r.content, 'html.parser')
                    table = soup.find('table')
            df2 = pd.read_html(str(table))[0]
            df2 = df2.assign(Year = row['Year'],Team = row['Team'],ForBR = row['ForBR'])
            df_new = pd.concat([df_new,df2])
            sleep(3)
    except:
        pass
df_new.head(15)
df_new.to_csv('IndividualGamesData.csv')

In [11]:
#Drop double rows
df_new = df_new.droplevel(0, axis=1)

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [ ]:
#Change row labels
df_new.columns.values[[-1, -2,-3]] = ['ForBR', 'Team','Year']

In [ ]:
#Remove all rows not team totals
df_new = df_new.loc[df_new['Starters'] == 'Team Totals']

In [ ]:
#Tidy up unneeded columns
df_new = df_new.drop(columns = ['Starters','MP','+/-'])

# Merge back to original file

In [ ]:
Merged_df = pd.merge(df,df_new, how = 'left', on = ['ForBR','Team'])

In [15]:
Merged_df = pd.read_csv('Merged_DataSet.csv')
Merged_df

,Unnamed: 0,Unnamed: 0.1,Start (ET),Away?,Opponent,TeamPts,OppPts,Team,TeamName,Location,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_y
0,0,0,19:00:00,@,Charlotte Hornets,91.0,109.0,ATL,Atlanta Hawks,"[35.2272086, -80.8430827]",...,6,32,38,19,11,4,13,29,91,2017
1,1,0,19:00:00,@,Charlotte Hornets,91.0,109.0,ATL,Atlanta Hawks,"[35.2272086, -80.8430827]",...,6,32,38,19,11,4,13,29,91,2017
2,2,1,15:30:00,@,Brooklyn Nets,104.0,116.0,ATL,Atlanta Hawks,"[40.678177, -73.94416]",...,16,32,48,21,6,2,16,25,104,2017
3,3,1,15:30:00,@,Brooklyn Nets,104.0,116.0,ATL,Atlanta Hawks,"[40.678177, -73.94416]",...,16,32,48,21,6,2,16,25,104,2017
4,4,2,19:30:00,@,Miami Heat,93.0,104.0,ATL,Atlanta Hawks,"[25.7741728, -80.19362]",...,6,35,41,14,9,1,20,17,93,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19435,19435,9717,19:00:00,NaN,San Antonio Spurs,112.0,129.0,WAS,Washington Wizards,"[38.8950368, -77.0365427]",...,9,25,34,22,2,4,9,16,112,2019
19436,19436,9718,19:30:00,@,New York Knicks,110.0,113.0,WAS,Washington Wizards,"[40.75054931640625, -73.99356079101562]",...,21,29,50,24,12,4,8,16,110,2019
19437,19437,9718,19:30:00,@,New York Knicks,110.0,113.0,WAS,Washington Wizards,"[40.75054931640625, -73.99356079101562]",...,21,29,50,24,12,4,8,16,110,2019
19438,19438,9719,19:00:00,NaN,Boston Celtics,110.0,116.0,WAS,Washington Wizards,"[38.8950368, -77.0365427]",...,13,45,58,28,2,3,15,18,110,2019


In [16]:
Merged_df = Merged_df.drop_duplicates(subset=['Team','ForBR'])
Merged_df

,Unnamed: 0,Unnamed: 0.1,Start (ET),Away?,Opponent,TeamPts,OppPts,Team,TeamName,Location,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_y
0,0,0,19:00:00,@,Charlotte Hornets,91.0,109.0,ATL,Atlanta Hawks,"[35.2272086, -80.8430827]",...,6,32,38,19,11,4,13,29,91,2017
2,2,1,15:30:00,@,Brooklyn Nets,104.0,116.0,ATL,Atlanta Hawks,"[40.678177, -73.94416]",...,16,32,48,21,6,2,16,25,104,2017
4,4,2,19:30:00,@,Miami Heat,93.0,104.0,ATL,Atlanta Hawks,"[25.7741728, -80.19362]",...,6,35,41,14,9,1,20,17,93,2017
6,6,3,20:00:00,@,Chicago Bulls,86.0,91.0,ATL,Atlanta Hawks,"[41.8810409, -87.6711385]",...,7,33,40,22,10,5,10,23,86,2017
8,8,4,19:30:00,NaN,Denver Nuggets,100.0,105.0,ATL,Atlanta Hawks,"[33.7489924, -84.3902644]",...,11,32,43,23,7,4,12,21,100,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19106,19106,9553,20:00:00,@,Denver Nuggets,95.0,90.0,WAS,Washington Wizards,"[39.7392364, -104.9848623]",...,9,45,54,24,9,6,10,21,95,2019
19108,19108,9554,19:00:00,NaN,Chicago Bulls,114.0,115.0,WAS,Washington Wizards,"[38.8950368, -77.0365427]",...,9,30,39,24,8,9,12,18,114,2019
19110,19110,9555,19:00:00,NaN,San Antonio Spurs,112.0,129.0,WAS,Washington Wizards,"[38.8950368, -77.0365427]",...,9,25,34,22,2,4,9,16,112,2019
19112,19112,9556,19:30:00,@,New York Knicks,110.0,113.0,WAS,Washington Wizards,"[40.75054931640625, -73.99356079101562]",...,21,29,50,24,12,4,8,16,110,2019


In [ ]:
#Export
Merged_df.to_csv('Merged_DataSet.csv')
Merged_df